# create_temp_reports.

In [1]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import numpy as np
import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db
from scripts.normalize_taxa import add_normalized_name_column

from scripts.normalize_data import (
    check_duplicate_columns
)

In [2]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


date = '2022-04-28'


taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

PI_4_file = OUTPUT_DIR /'taxa'/'LIMS'/f'PI_Micropal_CSV_4_normalized_taxa_list_with_pbdb_{date}.csv'
additional_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/'addtional_species.csv'
PI_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'


## create a file with all columns  in LIMS 4

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [14]:
data = []
for path in metadata['path']:
    if '317_U1351_planktic_forams.csv' in str(path):
        header = 1
    else:
        header = 0
    df = pd.read_csv(clean_data_path/path, dtype=str, header=header)
    df.dropna(axis="columns", how='all', inplace=True)
    for index, row in df.iterrows():
        row_data = {"path": path}
        for col in df.columns:
            row_data[col] = row[col]
        data.append(row_data)

In [15]:
len(data)

6818

In [16]:
new_df = pd.DataFrame(data)
new_df.shape

(6818, 1712)

In [17]:
new_df = new_df[sorted(new_df.columns)]


In [18]:
new_df.to_csv('../../output/tmp/combine_mircopal_4.csv')